## Experiments on Patches of Confocal tyx-Image Series

**Author**: Prisca Dotti

**Last Edit**: 17.06.2024

This notebook is used to run some experiments using the U-Net model on patches obtained from either real or fake confocal imaging data.

The dataset used for training could be one of the following:
- dataset of patches extracted in a meaningful way from confocal imaging recordings
- dataset of recordings from which patches are extracted when processing them in the U-Net for memory management reasons and which are recombined as whole movies after inference
- dataset of simulated confocal imaging patches --> this is similar to what I've been doing so far
- a combination of real and fake patches of confocal imaging data

In [1]:
# autoreload is used to reload modules automatically before entering the
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# To import modules from parent directory in Jupyter Notebook
import sys

sys.path.append("..")

In [2]:
import logging
import os
import random

import numpy as np
import torch
import wandb
from torch import nn, optim

# from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter

from config import TrainingConfig, config
from data.datasets import PatchCaEventsDataset, PatchSparksDataset
from models.UNet import unet
from utils.training_inference_tools import (
    MyTrainingManager,
    TransformedSparkDataset,
    random_flip,
    random_flip_noise,
    sampler,
    test_function_patches,
    training_step,
    weights_init,
)
from utils.training_script_utils import (
    get_sample_ids,
    init_config_file_path,
    init_criterion,
    init_dataset,
    init_model,
)


logger = logging.getLogger(__name__)

In [3]:
logger.setLevel(logging.DEBUG)

In [4]:
##################### Get training-specific parameters #####################

# Initialize training-specific parameters
# (get the configuration file path from ArgParse)
config_filename = os.path.join("config_files", "config_sparks_patches.ini")
# config_filename = os.path.join("config_files", "config_final_model.ini")

params = TrainingConfig(training_config_file=config_filename)

# Print parameters to console if needed
params.print_params()

######################### Initialize random seeds ##########################

# We used these random seeds to ensure reproducibility of the results

torch.manual_seed(0)  # <--------------------------------------------------!
random.seed(0)  # <--------------------------------------------------------!
np.random.seed(0)  # <-----------------------------------------------------!

[10:12:09] [  INFO  ] [   config   ] <318 > -- Loading C:\Users\dotti\Code\sparks_project\config_files\config_sparks_patches.ini
[10:12:09] [  INFO  ] [   config   ] <570 > --     training_config_file: C:\Users\dotti\Code\sparks_project\config_files\config_sparks_patches.ini
[10:12:09] [  INFO  ] [   config   ] <570 > --              dataset_dir: C:\Users\dotti\Code\sparks_project\data\sparks_dataset
[10:12:09] [  INFO  ] [   config   ] <570 > --                        c: <configparser.ConfigParser object at 0x0000020B84B8FAF0>
[10:12:09] [  INFO  ] [   config   ] <570 > --                 run_name: sparks_patches
[10:12:09] [  INFO  ] [   config   ] <570 > --            load_run_name: 
[10:12:09] [  INFO  ] [   config   ] <570 > --               load_epoch: 0
[10:12:09] [  INFO  ] [   config   ] <570 > --             train_epochs: 100000
[10:12:09] [  INFO  ] [   config   ] <570 > --                criterion: lovasz_softmax
[10:12:09] [  INFO  ] [   config   ] <570 > --               

In [5]:
# params.set_device("cpu")
params.display_device_info()

[10:12:09] [  INFO  ] [   config   ] <566 > -- Using cuda


In [6]:
############################ Configure datasets ############################

# Select samples for training and testing based on dataset size
train_sample_ids = get_sample_ids(
    train_data=True,
    dataset_size=params.dataset_size,
)
test_sample_ids = get_sample_ids(
    train_data=False,
    dataset_size=params.dataset_size,
)

# Initialize training dataset
dataset = PatchSparksDataset(
    params=params,
    base_path=params.dataset_dir,
    sample_ids=train_sample_ids,
    load_instances=True,  # this is needed to detect patches wrt spark peaks
    inference=None,
)

# Apply transforms based on noise_data_augmentation setting
# (transforms are applied when getting a sample from the dataset)
transforms = random_flip_noise if params.noise_data_augmentation else random_flip
dataset = TransformedSparkDataset(dataset, transforms)

logger.info(f"Samples in dataset (patches): {len(dataset)}")

# Initialize testing datasets
testing_dataset = PatchSparksDataset(
    params=params,
    base_path=params.dataset_dir,
    sample_ids=test_sample_ids,
    load_instances=True,  # this is needed to detect patches wrt spark peaks
    inference=None,
)

logger.info(f"Samples in test dataset (patches): {len(testing_dataset)}")

[10:16:25] [ DEBUG  ] [data.datasets] <1278> -- Getting patches from movie 1/34
[10:16:25] [ DEBUG  ] [data.datasets] <1304> --   Removing patch (214, 3, 195) due to puff or wave pixels
[10:16:25] [ DEBUG  ] [data.datasets] <1304> --   Removing patch (302, 0, 192) due to puff or wave pixels
[10:16:25] [ DEBUG  ] [data.datasets] <1304> --   Removing patch (259, 0, 213) due to puff or wave pixels
[10:16:25] [ DEBUG  ] [data.datasets] <1278> -- Getting patches from movie 2/34
[10:16:25] [ DEBUG  ] [data.datasets] <1304> --   Removing patch (297, 29, 305) due to puff or wave pixels
[10:16:25] [ DEBUG  ] [data.datasets] <1278> -- Getting patches from movie 3/34
[10:16:25] [ DEBUG  ] [data.datasets] <1304> --   Removing patch (152, 27, 368) due to puff or wave pixels
[10:16:25] [ DEBUG  ] [data.datasets] <1304> --   Removing patch (321, 4, 321) due to puff or wave pixels
[10:16:25] [ DEBUG  ] [data.datasets] <1304> --   Removing patch (382, 17, 283) due to puff or wave pixels
[10:16:25] [ DE

In [7]:
# import napari

# viewer = napari.Viewer()

# train_sample_id = 9
# train_sample = dataset[train_sample_id]
# train_data = train_sample["data"].numpy()
# train_labels = train_sample["labels"].numpy()
# viewer.add_image(train_data)
# viewer.add_labels(train_labels)

# test_sample_id = 0
# test_sample = testing_dataset[test_sample_id]
# test_data = test_sample["data"].numpy()
# test_labels = test_sample["labels"].numpy()
# viewer.add_image(test_data)
# viewer.add_labels(test_labels)

In [8]:
# Initialize data loaders
dataset_loader = DataLoader(
    dataset,
    batch_size=params.batch_size,
    num_workers=params.num_workers,
    pin_memory=params.pin_memory,
)

In [9]:
############################## Configure UNet ##############################

# Initialize the UNet model
network = init_model(params=params)

# Move the model to the GPU if available
if params.device.type != "cpu":
    network = nn.DataParallel(network).to(params.device, non_blocking=True)
    # cudnn.benchmark = True

# Watch the model with wandb for logging if enabled
if params.wandb_log:
    wandb.watch(network)

# Initialize UNet weights if required
if params.initialize_weights:
    logger.info("Initializing UNet weights...")
    network.apply(weights_init)

# The following line is commented as it does not work on Windows
# torch.compile(network, mode="default", backend="inductor")

In [10]:
########################### Initialize training ############################

# Initialize the optimizer based on the specified type
if params.optimizer == "adam":
    optimizer = optim.Adam(network.parameters(), lr=params.lr_start)
elif params.optimizer == "adadelta":
    optimizer = optim.Adadelta(network.parameters(), lr=params.lr_start)
elif params.optimizer == "sgd":
    optimizer = optim.SGD(network.parameters(), lr=params.lr_start)
else:
    logger.error(f"{params.optimizer} is not a valid optimizer.")
    exit()

# Initialize the learning rate scheduler if specified
if params.scheduler == "step":
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=params.scheduler_step_size,
        gamma=params.scheduler_gamma,
    )
else:
    scheduler = None

# Define the output directory path
output_path = os.path.join(config.output_dir, params.run_name)
logger.info(f"Output directory: {os.path.realpath(output_path)}")

# Initialize the summary writer for TensorBoard logging
summary_writer = SummaryWriter(os.path.join(output_path, "summary"), purge_step=0)

# Check if a pre-trained model should be loaded
if params.load_run_name != "":
    load_path = os.path.join(config.output_dir, params.load_run_name)
    logger.info(f"Model loaded from directory: {os.path.realpath(load_path)}")
else:
    load_path = None

# Initialize the loss function
criterion = init_criterion(params=params, dataset=dataset)

# Create a directory to save predicted class movies
preds_output_dir = os.path.join(output_path, "predictions")
os.makedirs(preds_output_dir, exist_ok=True)

# Create a dictionary of managed objects
managed_objects = {"network": network, "optimizer": optimizer}
if scheduler is not None:
    managed_objects["scheduler"] = scheduler

# Create a training manager with the specified training and testing functions
trainer = MyTrainingManager(
    # Training parameters
    training_step=lambda _: training_step(
        dataset_loader=dataset_loader,
        params=params,
        sampler=sampler,
        network=network,
        optimizer=optimizer,
        criterion=criterion,
        scheduler=scheduler,
        # scaler=GradScaler(),
    ),
    save_every=params.c.getint("training", "save_every", fallback=5000),
    load_path=load_path,
    save_path=output_path,
    managed_objects=unet.managed_objects(managed_objects),
    # Testing parameters
    test_function=lambda _: test_function_patches(
        network=network,
        device=params.device,
        criterion=criterion,
        params=params,
        testing_dataset=testing_dataset,
        training_name=params.run_name,
        output_dir=preds_output_dir,
        training_mode=True,
    ),
    test_every=params.c.getint("training", "test_every", fallback=1000),
    plot_every=params.c.getint("training", "test_every", fallback=1000),
    summary_writer=summary_writer,
)

# Load the model if a specific epoch is provided
if params.load_epoch != 0:
    trainer.load(params.load_epoch)

[10:17:06] [  INFO  ] [  __main__  ] < 26 > -- Output directory: C:\Users\dotti\Code\sparks_project\models\saved_models\sparks_patches


In [ ]:
# Validate the network before training if resuming from a checkpoint
# if params.load_epoch > 0:
#     logger.info("Validate network before training")
#     trainer.run_validation(wandb_log=params.wandb_log)

In [ ]:
# Set the network in training mode
network.train()

# Train the model for the specified number of epochs
logger.info("Starting training")
trainer.train(
    params.train_epochs,
    print_every=params.c.getint("training", "print_every", fallback=100),
    wandb_log=params.wandb_log,
)

In [12]:
logger.info("Starting final validation")
# Run the final validation/testing procedure
trainer.run_validation(wandb_log=params.wandb_log)

[10:18:04] [  INFO  ] [  __main__  ] < 1  > -- Starting final validation
[10:18:04] [  INFO  ] [utils.training_inference_tools] <1486> -- Validating network at iteration 0...
[10:18:04] [ DEBUG  ] [utils.training_inference_tools] <1205> -- Test function: running samples in UNet
[10:18:05] [ DEBUG  ] [utils.training_inference_tools] <1233> -- Time to run testing dataset in UNet: 1.12 s
[10:18:05] [ DEBUG  ] [utils.training_inference_tools] <1235> -- Test function: computing loss
[10:18:06] [ DEBUG  ] [utils.training_inference_tools] <1255> -- Processing patch 0/200
[10:18:06] [ DEBUG  ] [utils.training_inference_tools] <1279> -- Test function: re-organising annotations
[10:18:06] [ DEBUG  ] [utils.training_inference_tools] <1289> -- Time to re-organise annotations: 0.00 s
[10:18:06] [ DEBUG  ] [utils.training_inference_tools] <1294> -- Test function: getting processed output (segmentation and instances)
[10:18:06] [ DEBUG  ] [utils.training_inference_tools] <1311> -- Time to process pre

In [ ]:
# Close the summary writer
summary_writer.close()

# Close the wandb run
if params.wandb_log:
    wandb.finish()